In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm

import gym

from stable_baselines3.ppo import PPO

from cppo import CPPO_Policy
from env import SlindingAntEnv

In [2]:
env = SlindingAntEnv(3)

In [3]:
ppo = PPO(CPPO_Policy, env, 
        policy_kwargs={'net_arch': [{'pi': [256, 256], 'vf':[256, 256]}], 
                       'activation_fn': nn.Tanh, 
                       'optimizer_kwargs':{'m':10}})

In [4]:
ppo.learn(total_timesteps=10000, progress_bar=True)

Output()